In [15]:
import pandas as pd
import numpy as np
import json
import os
from pathlib import Path

# Set paths
RAW_DATA_DIR = Path('../data/raw')
ADOLESCENT_DIR = RAW_DATA_DIR / 'hr_adolescent'
MIMIC_DIR = RAW_DATA_DIR / 'hr_mimic_iii'

# Create output directories
PROCESSED_DIR = Path('../data/processed')
ADOLESCENT_OUTPUT_DIR = PROCESSED_DIR / 'hr_adolescent'
MIMIC_OUTPUT_DIR = PROCESSED_DIR / 'hr_mimic_iii'

ADOLESCENT_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
MIMIC_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("✓ Directories created")

✓ Directories created


# Data Preparation

This notebook prepares the heart rate dataset from two sources:
1. **hr_adolescent**: Fitbit heart rate data from adolescent subjects
2. **hr_mimic_iii**: MIMIC-III clinical heart rate data

## Tasks:
- Split adolescent heart rate data by subject
- Create subject metadata JSON file
- Clean MIMIC-III data (remove empty rows and high-NaN columns)

## Part 1: Process Adolescent Heart Rate Data

### 1.1 Load and Split Heart Rate Data by Subject

In [16]:
# Load the heart rate data (large file - may take a moment)
hr_file = ADOLESCENT_DIR / 'Fitbit-heart-rate.txt'
print(f"Loading heart rate data from {hr_file}...")

# Read the large file
hr_data = pd.read_csv(hr_file)
print(f"✓ Loaded {len(hr_data):,} records")
print(f"  Columns: {list(hr_data.columns)}")
print(f"  Unique subjects: {hr_data['Subject code number'].nunique()}")

# Display sample data
hr_data.head()

Loading heart rate data from ..\data\raw\hr_adolescent\Fitbit-heart-rate.txt...
✓ Loaded 2,771,807 records
  Columns: ['Subject code number', 'Local datetime [ISO8601]', 'Local date [yyyy-mm-dd]', 'Local time [hh:mm]', 'UTC offset [hr]', 'heart rate [#/min]']
  Unique subjects: 24


,Subject code number,Local datetime [ISO8601],Local date [yyyy-mm-dd],Local time [hh:mm],UTC offset [hr],heart rate [#/min]
0,903,2019-10-15T00:00:00+0200,2019-10-15,00:00,2,64
1,903,2019-10-15T00:01:00+0200,2019-10-15,00:01,2,67
2,903,2019-10-15T00:02:00+0200,2019-10-15,00:02,2,66
3,903,2019-10-15T00:03:00+0200,2019-10-15,00:03,2,68
4,903,2019-10-15T00:04:00+0200,2019-10-15,00:04,2,66


In [17]:
# Split heart rate data by subject and save to individual CSV files
subject_codes = hr_data['Subject code number'].unique()
print(f"Splitting data for {len(subject_codes)} subjects...")

for subject_code in subject_codes:
    # Filter data for this subject
    subject_hr = hr_data[hr_data['Subject code number'] == subject_code]
    
    # Save to CSV
    output_file = ADOLESCENT_OUTPUT_DIR / f'subject_{subject_code}_hr.csv'
    subject_hr.to_csv(output_file, index=False)
    
    print(f"  ✓ Subject {subject_code}: {len(subject_hr):,} records → {output_file.name}")

print(f"\n✓ All heart rate files saved to {ADOLESCENT_OUTPUT_DIR}")

Splitting data for 24 subjects...
  ✓ Subject 903: 118,354 records → subject_903_hr.csv
  ✓ Subject 907: 108,150 records → subject_907_hr.csv
  ✓ Subject 911: 122,065 records → subject_911_hr.csv
  ✓ Subject 912: 118,311 records → subject_912_hr.csv
  ✓ Subject 914: 109,735 records → subject_914_hr.csv
  ✓ Subject 918: 119,580 records → subject_918_hr.csv
  ✓ Subject 919: 111,615 records → subject_919_hr.csv
  ✓ Subject 926: 123,220 records → subject_926_hr.csv
  ✓ Subject 929: 121,288 records → subject_929_hr.csv
  ✓ Subject 936: 113,827 records → subject_936_hr.csv
  ✓ Subject 941: 122,008 records → subject_941_hr.csv
  ✓ Subject 943: 116,360 records → subject_943_hr.csv
  ✓ Subject 944: 113,665 records → subject_944_hr.csv
  ✓ Subject 959: 116,882 records → subject_959_hr.csv
  ✓ Subject 962: 123,846 records → subject_962_hr.csv
  ✓ Subject 964: 102,367 records → subject_964_hr.csv
  ✓ Subject 967: 114,783 records → subject_967_hr.csv
  ✓ Subject 973: 124,691 records → subject_973_h

### 1.2 Create Subject Information JSON File

In [18]:
# Load population/subject information
population_file = ADOLESCENT_DIR / 'population.csv'
population_data = pd.read_csv(population_file)

print(f"✓ Loaded population data: {len(population_data)} subjects")
print(f"  Columns: {list(population_data.columns)}")

# Display sample
population_data.head()

✓ Loaded population data: 24 subjects
  Columns: ['Subject code number', 'Gender [M=male F=female]', 'Length [cm]', 'Weight [kg]', 'Age [yr]', 'HbA1C_1 [yyyy]', 'HbA1C_1 [mm]', 'HbA1C_1 [mmol/mol]', 'HbA1C_2 [yyyy]', 'HbA1C_2 [mm]', 'HbA1C_2 [mmol/mol]', 'Pump brand', 'Pump model', 'FGM/CGM brand', 'FGM/CGM model', 'Activity tracker brand', 'Activity tracker model']


,Subject code number,Gender [M=male F=female],Length [cm],Weight [kg],Age [yr],HbA1C_1 [yyyy],HbA1C_1 [mm],HbA1C_1 [mmol/mol],HbA1C_2 [yyyy],HbA1C_2 [mm],HbA1C_2 [mmol/mol],Pump brand,Pump model,FGM/CGM brand,FGM/CGM model,Activity tracker brand,Activity tracker model
0,903,F,154.5,42.7,10,2019,7,37,2019.0,11.0,42.0,Medtronic,MiniMed 640G,Abbott,Freestyle Libre,Fitbit,Inspire HR
1,907,F,168.2,58.7,11,2019,10,54,2020.0,1.0,NaN,Medtronic,MiniMed 640G,Abbott,Freestyle Libre,Fitbit,Inspire HR
2,911,F,172.0,59.8,15,2019,10,56,2020.0,1.0,64.0,Omnipod,NaN,Abbott,Freestyle Libre,Fitbit,Inspire HR
3,912,M,162.0,74.4,13,2019,6,53,2020.0,12.0,54.0,Medtronic,MiniMed 640G and MiniMed 670G,Abbott,Freestyle Libre,Fitbit,Inspire HR
4,914,M,186.6,68.5,13,2019,9,35,2019.0,11.0,38.0,Medtronic,MiniMed 640G,Abbott,Freestyle Libre,Fitbit,Inspire HR


In [19]:
# Extract only required fields: subject code, gender, length, weight, age
required_fields = {
    'Subject code number': 'subject_code',
    'Gender [M=male F=female]': 'gender',
    'Length [cm]': 'length_cm',
    'Weight [kg]': 'weight_kg',
    'Age [yr]': 'age_years'
}

# Select and rename columns
subjects_info = population_data[list(required_fields.keys())].copy()
subjects_info.rename(columns=required_fields, inplace=True)

# Convert to list of dictionaries
subjects_list = subjects_info.to_dict('records')

# Save as JSON
json_output_file = ADOLESCENT_OUTPUT_DIR / 'subjects_info.json'
with open(json_output_file, 'w') as f:
    json.dump(subjects_list, f, indent=2)

print(f"✓ Subject information saved to {json_output_file}")
print(f"  Total subjects: {len(subjects_list)}")
print(f"\nSample entry:")
print(json.dumps(subjects_list[0], indent=2))

✓ Subject information saved to ..\data\processed\hr_adolescent\subjects_info.json
  Total subjects: 24

Sample entry:
{
  "subject_code": 903,
  "gender": "F",
  "length_cm": 154.5,
  "weight_kg": 42.7,
  "age_years": 10
}


## Part 2: Clean MIMIC-III Data

### 2.1 Process All MIMIC-III Files

In [22]:
def clean_mimic_file(file_path, output_dir, nan_threshold=0.8):
    """
    Clean a MIMIC-III file by:
    1. Removing rows where all values are NaN
    2. Removing columns where >80% of values are NaN
    
    Args:
        file_path: Path to input CSV file
        output_dir: Directory to save cleaned file
        nan_threshold: Threshold for removing columns (default: 0.8 = 80%)
    
    Returns:
        Dictionary with cleaning statistics
    """
    # Load data
    df = pd.read_csv(file_path)
    original_rows = len(df)
    original_cols = len(df.columns)
    
    # Step 1: Remove rows where ALL values are NaN (excluding the first column)
    # Get all columns except the first one
    cols_to_check = df.columns[1:11]
    
    # Drop rows where all values in these columns are NaN
    df_cleaned = df.dropna(subset=cols_to_check, how='all')
    rows_after_cleaning = len(df_cleaned)
    removed_rows = original_rows - rows_after_cleaning
    
    # Step 2: Remove columns with >80% NaN values
    # Calculate NaN percentage for each column
    nan_percentage = df_cleaned.isna().sum() / len(df_cleaned)
    
    # Keep only columns with NaN percentage <= threshold
    cols_to_keep = nan_percentage[nan_percentage <= nan_threshold].index.tolist()
    df_cleaned = df_cleaned[cols_to_keep]
    
    cols_after_cleaning = len(df_cleaned.columns)
    removed_cols = original_cols - cols_after_cleaning
    
    # Save cleaned data
    output_file = output_dir / file_path.name
    df_cleaned.to_csv(output_file, index=False)
    
    return {
        'filename': file_path.name,
        'original_rows': original_rows,
        'cleaned_rows': rows_after_cleaning,
        'removed_rows': removed_rows,
        'original_cols': original_cols,
        'cleaned_cols': cols_after_cleaning,
        'removed_cols': removed_cols,
        'remaining_columns': list(df_cleaned.columns)
    }

print("✓ Cleaning function defined")

✓ Cleaning function defined


In [23]:
# Get all MIMIC-III CSV files
mimic_files = sorted(MIMIC_DIR.glob('*.csv'))
print(f"Found {len(mimic_files)} MIMIC-III files to process\n")

# Process each file
cleaning_stats = []

for file_path in mimic_files:
    print(f"Processing {file_path.name}...")
    stats = clean_mimic_file(file_path, MIMIC_OUTPUT_DIR, nan_threshold=0.9)
    cleaning_stats.append(stats)
    
    print(f"  Original: {stats['original_rows']:,} rows × {stats['original_cols']} cols")
    print(f"  Cleaned:  {stats['cleaned_rows']:,} rows × {stats['cleaned_cols']} cols")
    print(f"  Removed:  {stats['removed_rows']:,} rows, {stats['removed_cols']} cols")
    print(f"  Remaining columns: {', '.join(stats['remaining_columns'][:5])}" + 
          (f", ... (+{len(stats['remaining_columns'])-5} more)" if len(stats['remaining_columns']) > 5 else ""))
    print()

print(f"✓ All files cleaned and saved to {MIMIC_OUTPUT_DIR}")

Found 17 MIMIC-III files to process

Processing adult_3016412n.csv...
  Original: 588,406 rows × 19 cols
  Cleaned:  579,065 rows × 18 cols
  Removed:  9,341 rows, 1 cols
  Remaining columns: Elapsed_Time_Sec, HR, PULSE, ABP Sys, ABP Dias, ... (+13 more)

Processing adult_3190202n.csv...
  Original: 2,645 rows × 48 cols
  Cleaned:  2,611 rows × 44 cols
  Removed:  34 rows, 4 cols
  Remaining columns: Elapsed_Time_Sec, HR, PULSE, RESP, ST-III, ... (+39 more)

Processing adult_3297027n.csv...
  Original: 49,940 rows × 14 cols
  Cleaned:  45,958 rows × 13 cols
  Removed:  3,982 rows, 1 cols
  Remaining columns: Elapsed_Time_Sec, HR, PULSE, CVP, NBP Sys, ... (+8 more)

Processing adult_3362386n.csv...
  Original: 2,639 rows × 47 cols
  Cleaned:  2,563 rows × 44 cols
  Removed:  76 rows, 3 cols
  Remaining columns: Elapsed_Time_Sec, HR, PULSE, RESP, ST-III, ... (+39 more)

Processing adult_3570073n.csv...
  Original: 2,970 rows × 53 cols
  Cleaned:  2,967 rows × 10 cols
  Removed:  3 rows, 

### 2.2 Summary Statistics

In [24]:
# Create summary dataframe
summary_df = pd.DataFrame(cleaning_stats)

# Calculate totals and averages
total_original_rows = summary_df['original_rows'].sum()
total_cleaned_rows = summary_df['cleaned_rows'].sum()
total_removed_rows = summary_df['removed_rows'].sum()

avg_original_cols = summary_df['original_cols'].mean()
avg_cleaned_cols = summary_df['cleaned_cols'].mean()

print("=" * 70)
print("MIMIC-III CLEANING SUMMARY")
print("=" * 70)
print(f"Total files processed: {len(mimic_files)}")
print(f"\nRows:")
print(f"  Original total:  {total_original_rows:,}")
print(f"  Cleaned total:   {total_cleaned_rows:,}")
print(f"  Removed total:   {total_removed_rows:,} ({total_removed_rows/total_original_rows*100:.1f}%)")
print(f"\nColumns (average):")
print(f"  Original:  {avg_original_cols:.1f}")
print(f"  Cleaned:   {avg_cleaned_cols:.1f}")
print(f"  Removed:   {avg_original_cols - avg_cleaned_cols:.1f}")
print("=" * 70)

# Display detailed summary
summary_df[['filename', 'cleaned_rows', 'cleaned_cols', 'removed_rows', 'removed_cols']]

MIMIC-III CLEANING SUMMARY
Total files processed: 17

Rows:
  Original total:  1,613,038
  Cleaned total:   1,574,801
  Removed total:   38,237 (2.4%)

Columns (average):
  Original:  25.6
  Cleaned:   19.5
  Removed:   6.1


,filename,cleaned_rows,cleaned_cols,removed_rows,removed_cols
0,adult_3016412n.csv,579065,18,9341,1
1,adult_3190202n.csv,2611,44,34,4
2,adult_3297027n.csv,45958,13,3982,1
3,adult_3362386n.csv,2563,44,76,3
4,adult_3570073n.csv,2967,10,3,43
5,adult_3618048n.csv,1520,44,0,7
6,adult_3722620n.csv,4308,47,1,21
7,adult_3864557n.csv,410013,16,3598,2
8,adult_3924317n.csv,1388,44,9,15
9,neonate_3182898n.csv,92720,8,3619,2


## Summary

### Output Files

#### Adolescent Data (hr_adolescent):
- **Heart Rate CSV Files**: Individual CSV files for each subject in `data/processed/hr_adolescent/`
  - Format: `subject_<code>_hr.csv`
- **Heart Rate TXT Files**: Text format for prompts in `data/processed/hr_adolescent/`
  - Format: `subject_<code>_hr.txt`
  - Example: `Time: 00:00 - heart rate [#/min]: 64`
- **Metadata**: `subjects_info.json` containing subject information (code, gender, length, weight, age)

#### MIMIC-III Data (hr_mimic_iii):
- **Cleaned CSV Files**: All MIMIC-III files cleaned and saved in `data/processed/hr_mimic_iii/`
  - Removed rows with all NaN values (excluding first column)
  - Removed columns with >80% NaN values
- **Text Files**: Prompt-ready text format in `data/processed/hr_mimic_iii/`
  - Example: `Time: 31s - HR: 117, Rhythm Status: 63005`
  - Only non-null values are included in each line

### Text Format Details
Each text file contains one line per row with the format:
```
Time: <time_value> - Feature1: value1, Feature2: value2, ...
```
- Only features with non-null values are included
- Perfect for creating prompts for LLM-based analysis
- Time format varies by dataset (timestamp for adolescent, elapsed seconds for MIMIC-III)

All processed data is ready for further analysis!

## Part 3: Create Text Versions for Prompts

### 3.1 Convert CSV Files to Text Format

In [26]:
def csv_to_text_format(csv_path, output_path, time_column=None):
    """
    Convert a CSV file to a text format suitable for prompts.
    Each row becomes a line with format: Time: XX - Feature1: value1, Feature2: value2, ...
    Only non-null values are included.
    
    Args:
        csv_path: Path to input CSV file
        output_path: Path to output text file
        time_column: Name of the time column (if None, uses row index)
    
    Returns:
        Number of lines written
    """
    # Read CSV
    df = pd.read_csv(csv_path)
    
    lines = []
    
    for idx, row in df.iterrows():
        # Start building the line
        if time_column and time_column in df.columns and pd.notna(row[time_column]):
            # Use the actual time column value
            time_value = row[time_column]
            line_parts = [f"Time: {time_value}"]
        else:
            # Use row index or elapsed time
            if 'Elapsed_Time_Sec' in df.columns and pd.notna(row['Elapsed_Time_Sec']):
                line_parts = [f"Time: {int(row['Elapsed_Time_Sec'])}s"]
            else:
                line_parts = [f"Time: {idx}"]
        
        # Add all non-null features (excluding the time column)
        for col in df.columns:
            # Skip time columns and any column we already used
            if col in ['Elapsed_Time_Sec', time_column, 'Subject code number', 
                       'Local datetime [ISO8601]', 'Local date [yyyy-mm-dd]', 
                       'Local time [hh:mm]', 'UTC offset [hr]']:
                continue
            
            # Only add if value is not null
            if pd.notna(row[col]):
                value = row[col]
                # Format the value appropriately
                if isinstance(value, float):
                    # Round to reasonable precision
                    if value.is_integer():
                        line_parts.append(f"{col}: {int(value)}")
                    else:
                        line_parts.append(f"{col}: {value:.2f}")
                else:
                    line_parts.append(f"{col}: {value}")
        
        # Join all parts with " - " for first separator, then ", " for features
        if len(line_parts) > 1:
            line = line_parts[0] + " - " + ", ".join(line_parts[1:])
            lines.append(line)
    
    # Write to file
    with open(output_path, 'w') as f:
        f.write('\n'.join(lines))
    
    return len(lines)

print("✓ Text conversion function defined")

✓ Text conversion function defined


### 3.2 Process Adolescent Files

In [27]:
# Create text versions of adolescent heart rate files
adolescent_csv_files = sorted(ADOLESCENT_OUTPUT_DIR.glob('subject_*_hr.csv'))
print(f"Converting {len(adolescent_csv_files)} adolescent files to text format...\n")

for csv_file in adolescent_csv_files:
    # Create output filename (replace .csv with .txt)
    txt_file = csv_file.with_suffix('.txt')
    
    # Convert using the local time column
    num_lines = csv_to_text_format(csv_file, txt_file, time_column='Local time [hh:mm]')
    
    print(f"  ✓ {csv_file.name} → {txt_file.name} ({num_lines:,} lines)")

print(f"\n✓ All adolescent files converted to text format")

Converting 24 adolescent files to text format...

  ✓ subject_903_hr.csv → subject_903_hr.txt (118,354 lines)
  ✓ subject_907_hr.csv → subject_907_hr.txt (108,150 lines)
  ✓ subject_911_hr.csv → subject_911_hr.txt (122,065 lines)
  ✓ subject_912_hr.csv → subject_912_hr.txt (118,311 lines)
  ✓ subject_914_hr.csv → subject_914_hr.txt (109,735 lines)
  ✓ subject_918_hr.csv → subject_918_hr.txt (119,580 lines)
  ✓ subject_919_hr.csv → subject_919_hr.txt (111,615 lines)
  ✓ subject_926_hr.csv → subject_926_hr.txt (123,220 lines)
  ✓ subject_929_hr.csv → subject_929_hr.txt (121,288 lines)
  ✓ subject_936_hr.csv → subject_936_hr.txt (113,827 lines)
  ✓ subject_941_hr.csv → subject_941_hr.txt (122,008 lines)
  ✓ subject_943_hr.csv → subject_943_hr.txt (116,360 lines)
  ✓ subject_944_hr.csv → subject_944_hr.txt (113,665 lines)
  ✓ subject_959_hr.csv → subject_959_hr.txt (116,882 lines)
  ✓ subject_962_hr.csv → subject_962_hr.txt (123,846 lines)
  ✓ subject_964_hr.csv → subject_964_hr.txt (102,3

### 3.3 Process MIMIC-III Files

In [28]:
# Create text versions of MIMIC-III files
mimic_csv_files = sorted(MIMIC_OUTPUT_DIR.glob('*.csv'))
print(f"Converting {len(mimic_csv_files)} MIMIC-III files to text format...\n")

for csv_file in mimic_csv_files:
    # Create output filename (replace .csv with .txt)
    txt_file = csv_file.with_suffix('.txt')
    
    # Convert (MIMIC uses Elapsed_Time_Sec)
    num_lines = csv_to_text_format(csv_file, txt_file)
    
    print(f"  ✓ {csv_file.name} → {txt_file.name} ({num_lines:,} lines)")

print(f"\n✓ All MIMIC-III files converted to text format")

Converting 17 MIMIC-III files to text format...

  ✓ adult_3016412n.csv → adult_3016412n.txt (579,065 lines)
  ✓ adult_3190202n.csv → adult_3190202n.txt (2,611 lines)
  ✓ adult_3297027n.csv → adult_3297027n.txt (45,958 lines)
  ✓ adult_3362386n.csv → adult_3362386n.txt (2,563 lines)
  ✓ adult_3570073n.csv → adult_3570073n.txt (2,967 lines)
  ✓ adult_3618048n.csv → adult_3618048n.txt (1,520 lines)
  ✓ adult_3722620n.csv → adult_3722620n.txt (4,308 lines)
  ✓ adult_3864557n.csv → adult_3864557n.txt (410,013 lines)
  ✓ adult_3924317n.csv → adult_3924317n.txt (1,388 lines)
  ✓ neonate_3182898n.csv → neonate_3182898n.txt (92,720 lines)
  ✓ neonate_3220897n.csv → neonate_3220897n.txt (38,238 lines)
  ✓ neonate_3306334n.csv → neonate_3306334n.txt (180,932 lines)
  ✓ neonate_3479334n.csv → neonate_3479334n.txt (574 lines)
  ✓ neonate_3632888n.csv → neonate_3632888n.txt (96,164 lines)
  ✓ neonate_3716249n.csv → neonate_3716249n.txt (9,645 lines)
  ✓ neonate_3875566n.csv → neonate_3875566n.txt (

### 3.4 Sample Text Output

In [29]:
# Display sample from an adolescent file
sample_adolescent = sorted(ADOLESCENT_OUTPUT_DIR.glob('subject_*_hr.txt'))[0]
print(f"Sample from {sample_adolescent.name}:")
print("=" * 70)
with open(sample_adolescent, 'r') as f:
    lines = f.readlines()
    for line in lines[:10]:  # Show first 10 lines
        print(line.rstrip())
print("...")
print("=" * 70)

# Display sample from a MIMIC file
print(f"\n")
sample_mimic = sorted(MIMIC_OUTPUT_DIR.glob('*.txt'))[0]
print(f"Sample from {sample_mimic.name}:")
print("=" * 70)
with open(sample_mimic, 'r') as f:
    lines = f.readlines()
    # Find lines with actual data (skip initial empty lines)
    data_lines = [l for l in lines if ' - ' in l]
    for line in data_lines[:10]:  # Show first 10 data lines
        print(line.rstrip())
print("...")
print("=" * 70)

Sample from subject_903_hr.txt:
Time: 00:00 - heart rate [#/min]: 64
Time: 00:01 - heart rate [#/min]: 67
Time: 00:02 - heart rate [#/min]: 66
Time: 00:03 - heart rate [#/min]: 68
Time: 00:04 - heart rate [#/min]: 66
Time: 00:05 - heart rate [#/min]: 66
Time: 00:06 - heart rate [#/min]: 65
Time: 00:07 - heart rate [#/min]: 67
Time: 00:08 - heart rate [#/min]: 66
Time: 00:09 - heart rate [#/min]: 67
...


Sample from adult_3016412n.txt:
Time: 31s - HR: 117, Rhythm Status: 63005
Time: 32s - HR: 117, Rhythm Status: 63005
Time: 33s - HR: 118, Rhythm Status: 63005
Time: 34s - HR: 115, Rhythm Status: 63005
Time: 42s - HR: 110, PVC Rate per Minute: 0, Rhythm Status: 63004
Time: 43s - HR: 112, PVC Rate per Minute: 0, Rhythm Status: 63004
Time: 44s - HR: 116, PVC Rate per Minute: 0, Rhythm Status: 63004
Time: 45s - HR: 113, PVC Rate per Minute: 0, Rhythm Status: 63004
Time: 46s - HR: 110, PVC Rate per Minute: 0, Rhythm Status: 63004
Time: 47s - HR: 113, PVC Rate per Minute: 0, Rhythm Status: 63